# Business Problem

A bank requested a machine learning model that would produce results on whether loan applicants among the bank's customers would get the loan or not.

Data Quality                                                                                
Descriptive Statistics
Modelling
Evaluation

1. Data Quality
It is the dataset that contains information about customers and we will work on deriving useful insights from the information and predicting whether a new customer will accept a loan offer.

1 - age: The age of the customer, must be over 18(numeric)

2 - job: customer is profession(categorical)

admin.
blue-collar
entrepreneur
housemaid
management
self-employed
retired
services
student
technician
unemployed
unknown
3 - marital: marital status(categorical)

married
single
divorced Note: 'divorced' means divorced or widowed
4 - education: educational status(categorical)

primary
secondary
tertiary
unknown
5 - balance: bank balance

6 - housing: has housing loan? (categorical: 'no','yes','unknown')

7 - campaign: number of contacts performed during this campaign and for this client (numeric)

8 - deposit: did the customer get a loan? (binary: 'yes', 'no')

9 - duration: contact time in seconds (numeric)

Libraries